
# IMPORTS


In [ ]:
!pip install --upgrade fastai
!pip install wget
!pip install torchsummary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tqdm
import h5py
import random
from prettytable import PrettyTable

import cv2
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from torchvision.utils import make_grid
import torch.nn.functional as F
import torch.optim as optim

import fastai
from fastai.vision import *
from fastai.data.core import DataLoaders
from fastai.basics import *
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.vision.augment import *
from fastai.vision import models
from fastai.vision.all import *
from fastai.layers import PixelShuffle_ICNR,ConvLayer,NormType

from sklearn.metrics import accuracy_score,max_error,explained_variance_score


# DATASET

In [ ]:
df=pd.read_csv("/content/drive/My Drive/DeepDrive/dataset.csv")

In [ ]:
class ImageDataset(Dataset):
  def __init__(self,df,image_size,tsfm):
    self.dataset=df.values
    self.image_size=image_size

  def __len__(self):
    #print(len(self.dataset))
    return len(self.dataset)

  def __getitem__(self,idx):
    img=cv2.imread(self.dataset[idx][0])
    msk=cv2.imread(self.dataset[idx][1])[:,:,0]
    #img=cv2.resize(img,(self.image_size,self.image_size))
    #msk=cv2.resize(msk,(self.image_size,self.image_size))
    img=np.transpose(img,(2,0,1))
    #X = tsfm(img)
    X=torch.cuda.FloatTensor(img)
    y=torch.cuda.LongTensor(msk)
    return X,y

In [ ]:
tsfm = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                          ])

In [ ]:
train_df=df[0:3000]
valid_df=df[28000:]
train_dataset=ImageDataset(df=train_df,image_size=224,tsfm=tsfm)
valid_dataset=ImageDataset(df=valid_df,image_size=224,tsfm=tsfm)

# LOSS

In [ ]:
def cse_loss(predictions,target):
  cse_loss=nn.CrossEntropyLoss()
  print(predictions.shape)
  print(target.shape)
  loss = cse_loss(predictions, target)
  return loss

# ACCURACY METRIC

In [ ]:
void_code = 0

def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

# DATA-LOADERS

In [ ]:
dls = DataLoaders.from_dsets(train_DepthDataset, valid_DepthDataset, bs=5, num_workers=0, shuffle=True)

# FASTAI D-UNET


In [ ]:
learn=unet_learner(dls, arch=resnet34,loss_func=cse_loss,metrics=acc_camvid, pretrained=True,n_in=3, n_out=8,opt_func=Adam, self_attention=True)

In [ ]:
learn.model.load_state_dict(torch.load("/content/drive/My Drive/DeepDrive/89.pth"))

In [ ]:
learn.model.to("cuda:0")

# TRAINING

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(n_epoch=4,lr_max=0.00010964782268274575)

In [ ]:
torch.save(learn.model, "/content/drive/My Drive/DeepDrive/model.pth")
torch.save(learn.model.state_dict(),"/content/drive/My Drive/DeepDrive/91.pth")

# OUR MODEL

## ENCODER

In [ ]:
class res_block(nn.Module):
    def __init__(self,in_channels,out_channels,strides,identity_fn=None):
        super(res_block,self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=strides,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU()
        self.downsample=identity_fn
    def forward(self,x):
        identity=x
        x=self.conv1(x)
        x=self.relu(x)
        x=self.bn1(x)
        x=self.conv2(x)
        x=self.relu(x)
        x=self.bn2(x)
        
        if self.downsample!=None:
            identity=self.downsample(identity)
        x=torch.add(x,identity)
        x=self.relu(x)
        
        return x
        
            
class encoder(nn.Module):
    def __init__(self,res_block,layers,image_channels):
        super(encoder,self).__init__()
        self.conv=nn.Conv2d(image_channels,64, kernel_size=7, stride=2,padding=3,bias=False)
        self.bn=nn.BatchNorm2d(64)
        self.relu=nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layers=layers
        self.layer1=self.res_layer(res_block,count=self.layers[0],in_channels=64,out_channels=64,stride=1)
        self.layer2=self.res_layer(res_block,count=self.layers[1],in_channels=64,out_channels=128,stride=2)
        self.layer3=self.res_layer(res_block,count=self.layers[2],in_channels=128,out_channels=256,stride=2)
        self.layer4=self.res_layer(res_block,count=self.layers[3],in_channels=256,out_channels=512,stride=2)
    def forward(self,x):
        #print("input",x.shape)
        skip_outputs=[]
        in_channel=3
        x=self.conv(x)
        x=self.bn(x)
        x=self.relu(x)
        #print(x.shape)
        x_skip1=x
        x=self.maxpool(x)
        x=self.layer1(x)
        x_skip2=x
        x=self.layer2(x)
        x_skip3=x
        x=self.layer3(x)
        x_skip4=x
        x=self.layer4(x)
        return x,x_skip1,x_skip2,x_skip3,x_skip4
        
    def res_layer(self,res_block,count,in_channels,out_channels,stride=1):
        layers=[]
        if stride!=1:
            downsample=nn.Sequential(nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,padding=0,bias=False),
                                      nn.BatchNorm2d(out_channels))
            
        else:
            downsample=None
        layers.append(res_block(in_channels,out_channels,stride,downsample))
        
        for i in range(count-1):
            layers.append(res_block(out_channels,out_channels,1,None))
        return nn.Sequential(*layers)
            
        

## ATTENTION MODULE

In [ ]:
class attention(nn.Module):
    def __init__(self,in_channels):
        super(attention,self).__init__()
        self.relu=nn.ReLU()
        self.conv=nn.Conv2d(in_channels,1,kernel_size=1,stride=1,padding=0)
        self.sigmoid=nn.Sigmoid()
    def forward(self,x,g):
        x_=torch.add(x,g)
        x_=self.relu(x)
        x_=self.conv(x)
        x_=self.sigmoid(x)
        x=torch.mul(x,x_)
        return x

## UNET-BLOCK

In [ ]:
class unet_block(nn.Module):
    def __init__(self,in_channels,out_channels,skip_channels):
        super(unet_block,self).__init__()
        final_channel=out_channels+skip_channels
        self.Pixelshuf=PixelShuffle_ICNR(in_channels,out_channels)
        self.conv=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1)
        self.attention=attention(out_channels)
        self.up_conv1=nn.Conv2d(final_channel,final_channel,kernel_size=3,stride=1,padding=1)
        self.up_conv2=nn.Conv2d(final_channel,final_channel,kernel_size=3,stride=1,padding=1)
        self.relu=nn.ReLU()
        
    def forward(self,x,skip):
        x=self.Pixelshuf(x)
        x=self.conv(x)
        skip=self.attention(skip,x)
        x=torch.cat([x, skip],dim=1)
        x=self.up_conv1(x)
        x=self.relu(x)
        x=self.up_conv2(x)
        x=self.relu(x)
        return x

In [ ]:
class Net(nn.Module):
    def __init__(self,num_classes):
        super(Net, self).__init__()
        self.encoder = encoder(res_block,[3,4,6,3],3)
        self.in_channels=1024
        self.bottle_conv1=nn.Conv2d(512,1024,kernel_size=3,stride=1,padding=1)
        self.bottle_conv2=nn.Conv2d(1024,512,kernel_size=3,stride=1,padding=1)
        self.relu=nn.ReLU()
        self.unet_block1=unet_block(1024,256,256)
        self.unet_block2=unet_block(512,128,128)
        self.unet_block3=unet_block(256,64,64)
        self.unet_block4=unet_block(128,64,64)
        self.conv_transpose=nn.ConvTranspose2d(128,128,kernel_size=2,stride=2,padding=0)
        self.out1=nn.Conv2d(128,num_classes,kernel_size=1,stride=1,padding=0)
        
    def forward(self,x):
        x_skip,skip1,skip2,skip3,skip4=self.encoder(x)
        x=self.bottle_conv1(x_skip)
        x=self.relu(x)
        x=self.bottle_conv2(x)
        x=self.relu(x)
        x=nn.ReLU()(torch.cat([x,x_skip],dim=1))
        x=self.unet_block1(x,skip4)
        x=self.unet_block2(x,skip3)
        x=self.unet_block3(x,skip2)
        x=self.unet_block4(x,skip1)
        x=self.conv_transpose(x)
        out1=self.out1(x)
        return out1


In [ ]:
encoder_decoder=Net(8)
encoder_decoder.to("cuda:0")
summary(encoder_decoder,(3,224,224),device="cuda")

## DATA-LOADER

In [ ]:
class Utils:
    @classmethod
    def build_target(cls,target,num_classes):
      target_map=np.transpose(to_categorical(target,num_classes=num_classes),axes=(0,3,1,2))
      return target_map


    @classmethod
    def get_loader(cls, image_size,batch_size,num_workers,CUDA):
        """Builds and returns Dataloader."""
        train_dataloader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=False)
        valid_dataloader = DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=False)
        return train_dataloader,valid_dataloader
    
    @classmethod
    def print_metrics(cls,avg_loss, avg_acc):
        t = PrettyTable(['Parameter', 'Value'])
        t.add_row(['Avg_Loss', avg_loss])
        t.add_row(['Avg_Acc', avg_acc])
        print(t)
        return
    
    @classmethod
    def show_batch(cls,dl,batch):
        j=0
        for images, masks in dl:
            j+=1
            images=images.view(batch*15,224,224,3).contiguous()
            masks=masks.view(batch*15,224,224).contiguous()
            #os.mkdir("batch_"+str(j))
            for i in range(batch*15):
                plt.imsave(os.path.join("batch_"+str(j),"batch_images"+str(i)+".png"),np.array(images[i].detach().numpy(),dtype='uint8'))
                plt.imsave(os.path.join("batch_"+str(j),"batch_masks"+str(i)+".png"),masks[i].detach().numpy())
        
            break
        return
    
    @classmethod
    def shut_down_decoder(cls,model,flag):
        i=0
        for parameter in model.parameters():
            parameter.requires_grad = flag
            i+=1
            if(i==108):
                break
        i=0
        for parameter in model.parameters():
            if parameter.requires_grad :
                print(i,"True",parameter.data.shape)
            else:
                print(i,"False",parameter.shape)
            i+=1
        return 
    

In [ ]:
encoder_decoder=Net(num_classes=8)
encoder_decoder.to("cuda:0")
#encoder_decoder.load_state_dict(torch.load("/content/drive/My Drive/encoder_decoder.pth"),strict=False)
Utils.shut_down_decoder(encoder_decoder,flag=False)

## LOSS AND METRICS

In [ ]:
#codes = np.loadtxt(path/'codes.txt', dtype=str)
#name2id = {v:k for k,v in enumerate(codes)}
void_code = 0

class Metrics:

    @classmethod
    def acc_camvid(cls,input, target):
      mask = target != void_code
      return (torch.argmax(input, dim=1)[mask]==target[mask]).float().mean()

    @classmethod
    def compute_loss(cls,predictions, targets):
        """Calculating the loss and metrics
        Args:
            prediction = predicted image
            target = Targeted image
        Output:
            loss : mse loss """
        ce_loss=torch.nn.CrossEntropyLoss()
        c_loss=ce_loss(predictions,targets)
        return c_loss
    
    @classmethod
    def approx_acc(cls,prediction,target):
        prediction = np.floor(prediction).astype(int).flatten()
        target = np.floor(target).astype(int).flatten()
        acc=accuracy_score(target,prediction)
        return acc
    
    @classmethod
    def max_error(cls,prediction,target):
        prediction = prediction.flatten()
        target = target.flatten()
        maxerror=max_error(target,prediction)
        return maxerror
    
    @classmethod
    def var_score(cls,target,prediction):
        var_score=explained_variance_score(target.flatten(),prediction.flatten())
        return var_score
        

## TRAINING

In [ ]:
class TrainModel:
    def __init__(self,batch_size=25,epochs=30,image_size=224,lr=0.001):
        self.batch_size=batch_size
        self.epochs=epochs
        self.image_size=image_size
        self.lr=lr
        self.CUDA = torch.cuda.is_available()
        self.device = "cuda:0"
        
    def fit(self,model):
        model = model.to(self.device)
        torch.save(model.state_dict(),"/content/drive/MyDrive/DeepDrive/our_model_1.pth")
        self.optimizer=torch.optim.Adam(model.parameters(), lr=self.lr)
        best_loss=0.7
        (train_dataloader,valid_dataloader)=Utils.get_loader(image_size=self.image_size,batch_size=self.batch_size,num_workers=0,CUDA=self.CUDA)
        print("Training")
        print(len(valid_dataloader))
        for epoch in tqdm.tqdm(range(self.epochs)):
            #Training Phase
            model.train()
            train_loss = []
            train_acc = []
            
            for i, (images, masks) in enumerate(train_dataloader):
                #images = images.view(images.shape[0],self.image_size,self.image_size,3)
                #masks = masks.view(masks.shape[0],self.image_size,self.image_size)
                images=images/255
                images.to(self.device)
                masks.to(self.device)
                self.optimizer.zero_grad()
                outputs = model(images)
                loss = Metrics.compute_loss(outputs,masks)
                print(loss)
                train_loss.append(loss.item())
                loss.backward()
                self.optimizer.step()
                acc=Metrics.acc_camvid(outputs,masks)
                train_acc.append(acc.item())
            #Validation Phase
            model.eval()
            valid_loss = []
            valid_acc = []
            #print("Validation")
            for i, (images, masks) in enumerate(valid_dataloader):

                """images = images.view(images.shape[0],self.image_size,self.image_size,3)
                masks = masks.view(masks.shape[0],self.image_size,self.image_size)"""
                images=images/255

                images.to(self.device)
                masks.to(self.device)
                self.optimizer.zero_grad()
                outputs = encoder_decoder(images)
                #print("outputs",outputs.shape)
               
                loss = Metrics.compute_loss(outputs,masks)
                #print("loss",loss,end=" ")
                valid_loss.append(loss.item())
                
                acc=Metrics.acc_camvid(outputs,masks)
                valid_acc.append(acc.item())
            
            avg_train_loss = np.average(train_loss)
            avg_train_acc = np.average(train_acc)
            Utils.print_metrics(avg_train_loss,avg_train_acc)
            
            avg_valid_loss = np.average(valid_loss)
            avg_valid_acc = np.average(valid_acc)
            Utils.print_metrics(avg_valid_loss,avg_valid_acc)
            
            
            if(avg_valid_loss<best_loss):
              torch.save(model.state_dict(),"/content/drive/MyDrive/DeepDrive/our_model_"+str(avg_valid_loss)+".pth")
              best_loss = avg_valid_loss
            
        return
  

In [ ]:
train=TrainModel(batch_size=10,epochs=5,image_size=224,lr=0.001)

In [ ]:
train.epochs=30
train.image_size=224
train.batch_size=12
train.lr=0.003
train.fit(encoder_decoder)